# Scout Mini Gazebo 仿真配置修改总结

本文档记录了对 Scout Mini 机器人 Gazebo 仿真配置的改进修改。

## 1. scout_mini.urdf.xacro

**文件路径:** `src/scout_ros/scout_description/urdf/scout_mini.urdf.xacro`

### 修改内容

| 修改项 | 原值 | 新值 | 说明 |
|--------|------|------|------|
| 添加 `base_footprint` 链接 | 无 | 新增 | ROS导航栈标准要求 |
| 激光雷达更新频率 | 10 Hz | 20 Hz | 提高扫描频率 |
| 激光雷达最大扫描距离 | 12.0 m | 6.0 m | 符合RPLIDAR A系列实际范围 |
| 激光雷达关节注释 | 无 | 新增 | 说明激光雷达位置用途 |
| 激光雷达链接注释 | 无 | 新增 | 增强代码可读性 |

### 新增/修改代码

**base_footprint 链接** (第22-28行)
```xml
<link name="base_footprint" />

<joint name="base_footprint_joint" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0" />
    <parent link="base_footprint" />
    <child link="base_link" />
</joint>
```

**激光雷达关节** (第90-96行，移除不必要的axis标签，添加注释)
```xml
<!-- 2D激光雷达(RPLIDAR)关节 -->
<!-- z=0.07m: 激光雷达安装在车体顶部，高于车体最高点 -->
<joint name="rplidar_joint" type="fixed">
    <origin xyz="0.0 0 0.07" rpy="0 0 0"/>
    <parent link="base_link"/>
    <child link="laser_link"/>
</joint>
```

**激光雷达扫描距离** (第136-140行)
```xml
<range>
    <min>0.15</min>
    <max>6.0</max>
    <resolution>0.01</resolution>
</range>
```

## 2. scout_mini.gazebo

**文件路径:** `src/scout_ros/scout_description/urdf/scout_mini.gazebo`

### 修改内容

| 修改项 | 原值 | 新值 | 说明 |
|--------|------|------|------|
| base_link 材料 | 注释状态 | `Gazebo/Grey` | 改善可视化效果 |
| laser_link 材料 | 无 | `Gazebo/Black` | 新增材料定义 |
| skid_steer_drive 插件 | 注释状态 | 启用 | 启用运动控制 |
| wheelSeparation | 4 (错误) | 0.416503 | 修正为实际轮距(track值) |
| robotBaseFrame | base_link | base_footprint | 修正TF发布 |

### 未修改的参数（保留官方原值）

| 参数 | 保留值 | 说明 |
|------|--------|------|
| wheelDiameter | 0.32918 | 官方测量的实际轮径，可能包含轮胎变形等因素 |
| torque | 1000 | 仿真中的扭矩参数，确保机器人能正常运动 |

### 新增材料定义
```xml
<gazebo reference="base_link">
    <material>Gazebo/Grey</material>
</gazebo>

<gazebo reference="laser_link">
    <material>Gazebo/Black</material>
</gazebo>
```

### 启用的 skid_steer_drive 插件配置
```xml
<gazebo>
    <plugin name="skid_steer_drive_controller" filename="libgazebo_ros_skid_steer_drive.so">
        <updateRate>100.0</updateRate>
        <robotNamespace></robotNamespace>
        <leftFrontJoint>front_left_wheel</leftFrontJoint>
        <rightFrontJoint>front_right_wheel</rightFrontJoint>
        <leftRearJoint>rear_left_wheel</leftRearJoint>
        <rightRearJoint>rear_right_wheel</rightRearJoint>
        <wheelSeparation>0.416503</wheelSeparation>
        <wheelDiameter>0.32918</wheelDiameter>
        <robotBaseFrame>base_footprint</robotBaseFrame>
        <torque>1000</torque>
        <commandTopic>cmd_vel</commandTopic>
        <broadcastTF>true</broadcastTF>
        <odometryTopic>odom</odometryTopic>
        <odometryFrame>odom</odometryFrame>
        <covariance_x>0.000100</covariance_x>
        <covariance_y>0.000100</covariance_y>
        <covariance_yaw>0.010000</covariance_yaw>
    </plugin>
</gazebo>
```

## 3. scout_mini_sim.launch

**文件路径:** `src/scout_ros/scout_description/launch/scout_mini_sim.launch`

### 修改内容

| 修改项 | 说明 |
|--------|------|
| 添加 map→odom 静态TF发布 | 完善TF树 |

### 新增代码
```xml
<!-- Static TF: map to odom -->
<node pkg="tf" type="static_transform_publisher" name="map_to_odom_tf"
      args="0 0 0 0 0 0 map odom 100" />
```

## 修改后的 TF 树结构

```
map
 └── odom                    (static_transform_publisher)
      └── base_footprint     (skid_steer_drive 插件)
           └── base_link     (robot_state_publisher)
                ├── laser_link
                ├── inertial_link
                ├── front_mount
                ├── rear_mount
                ├── front_left_wheel_link
                ├── front_right_wheel_link
                ├── rear_left_wheel_link
                └── rear_right_wheel_link
```

## 功能改进总结

1. **TF树完整** - 添加了 `base_footprint` 链接和 `map→odom` 变换
2. **支持 cmd_vel 控制** - 启用了 skid_steer_drive 插件，可通过 `/cmd_vel` 话题控制机器人
3. **发布里程计** - 在 `/odom` 话题发布里程计信息
4. **更高的激光扫描频率** - 从 10Hz 提升到 20Hz
5. **激光雷达配置优化** - 调整最大扫描距离为6.0m，符合实际硬件性能
6. **改善可视化** - 添加了 Gazebo 材料定义
7. **运动控制修复** - 通过轮子轴向补偿，解决左侧轮子反向问题

## 4. scout_mini_wheel.xacro

**文件路径:** `src/scout_ros/scout_description/urdf/scout_mini_wheel.xacro`

### 修改内容

为左侧轮子单独定义轴向，以补偿硬件上左侧轮子反向安装的问题

### 修改的代码 (第32-38行)

```xml
<!-- 左侧轮子(front_left和rear_left)轴向相反 -->
<xacro:if value="${wheel_prefix == 'front_left' or wheel_prefix == 'rear_left'}">
    <axis xyz="0 0 1" />
</xacro:if>
<xacro:unless value="${wheel_prefix == 'front_left' or wheel_prefix == 'rear_left'}">
    <axis xyz="0 0 -1" />
</xacro:unless>
```

### 原因

Scout Mini 的硬件设计中，左侧轮子（前左和后左）是反向安装的（见 scout_mini.urdf.xacro 第61-62行的注释）。
通过在宏定义中使用 xacro 的条件语句，为左右侧轮子定义不同的旋转轴向，在URDF层面解决了控制问题。

- 右侧轮子：轴向 `0 0 -1`（正向）
- 左侧轮子：轴向 `0 0 1`（反向补偿）

### 效果

修改后，发送向前命令时：
- 右侧轮子正向转动
- 左侧轮子反向转动（由于轴向反向）
- 最终结果：两侧轮子都向前，机器人直线前进

## 测试命令

```bash
# 启动仿真
roslaunch scout_description scout_mini_sim.launch

# 直线向前
rostopic pub /cmd_vel geometry_msgs/Twist -r 10 -- '[0.5, 0, 0]' '[0, 0, 0]'

# 逆时针旋转
rostopic pub /cmd_vel geometry_msgs/Twist -r 10 -- '[0, 0, 0]' '[0, 0, 0.5]'

# 顺时针旋转
rostopic pub /cmd_vel geometry_msgs/Twist -r 10 -- '[0, 0, 0]' '[0, 0, -0.5]'

# 向前并左转
rostopic pub /cmd_vel geometry_msgs/Twist -r 10 -- '[0.3, 0, 0]' '[0, 0, 0.3]'

# 停止
rostopic pub /cmd_vel geometry_msgs/Twist -r 10 -- '[0, 0, 0]' '[0, 0, 0]'

# 查看TF树
rosrun tf view_frames

# 查看里程计
rostopic echo /odom

# 查看激光数据
rostopic echo /scan
```